In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# import data

In [2]:
%load_ext memory_profiler

In [3]:
def other_ens():
    prct = 11
    df = pd.read_csv('n/Floor23_p11.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    #split train test and normalize data
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:, -1], test_size=0.3, shuffle=False)
    # Feature scaling
    sc = StandardScaler()
    x_train = sc.fit_transform(X_train.values)
    sc_t = StandardScaler()
    x_test = sc_t.fit_transform(X_test.values)  
    #init
    model1 = OneClassSVM(nu=0.5, kernel='linear', gamma="auto", shrinking=False)
    model2 = IsolationForest(n_estimators=80, max_features=0.2, contamination=0.1, random_state=42)
    model3 = LocalOutlierFactor(n_neighbors=20, contamination=0.1, novelty=True)
    #train
    start_time = time.time()
    preds_model1 = model1.fit_predict(x_train).reshape(-1, 1)
    after_code1_time = time.time()
    print(after_code1_time - start_time)
    preds_model2 = model2.fit_predict(x_train).reshape(-1, 1)    
    after_code2_time = time.time()
    print(after_code2_time - after_code1_time)
    model3.fit(x_train)
    preds_model3 = model3.predict(x_train).reshape(-1, 1)
    after_code3_time = time.time()
    print(after_code3_time - after_code2_time)
    
    # Stack the predictions horizontally
    X_meta = np.hstack((preds_model1, preds_model2, preds_model3))    
    # Assign labels based on the consensus of at least two base models
    x_predictions = (np.sum(X_meta > 0, axis=1) >= 1).astype(int) * 2 - 1
    
    #test
    ypreds_model1 = model1.predict(x_test).reshape(-1, 1)3
    ypreds_model2 = model2.predict(x_test).reshape(-1, 1)
    ypreds_model3 = model2.predict(x_test).reshape(-1, 1)
    
    # Stack the predictions horizontally
    stacked_X_test = np.hstack((ypreds_model1, ypreds_model2, ypreds_model3))
    
    # Assign labels based on the consensus of at least two base models
    y_predictions = (np.sum(stacked_X_test > 0, axis=1) >= 1).astype(int) * 2 - 1

In [ ]:
# Measure memory consumption
%memit -r 1 other_ens()

In [ ]:
# Measure CPU time
%timeit -r 1 -n 1 other_ens()

In [8]:
# Measure memory consumption
%memit -r 1 other_ens()

1624.1455099582672
3.815504550933838
20.75452995300293
peak memory: 648.61 MiB, increment: 486.95 MiB


In [9]:
# Measure CPU time
%timeit -r 1 -n 1 other_ens()

1514.5860681533813
2.639238119125366
12.909254550933838
29min 49s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
